In [ ]:
# Import all required libraries at the beginning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix, 
                             classification_report, roc_curve, auc)

import shap

plt.style.use('fivethirtyeight')
sns.set_palette("Set2")

In [ ]:

# https://www.kaggle.com/c/home-credit-default-risk/data
print("Loading application data...")

# Load data
app_data = pd.read_csv('application_train.csv')

print("Application Data Shape:", app_data.shape)
print("\nFirst few rows of Application Data:")
print(app_data.head())

print("\nDataset Info:")
print(app_data.info())

print("\nTarget Variable Distribution:")
print(app_data['TARGET'].value_counts())
print(f"Default Rate: {app_data['TARGET'].mean():.4f}")

missing_values = app_data.isnull().sum()
missing_percent = (missing_values / len(app_data)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percent})
missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values('Percentage', ascending=False)

print("\nColumns with missing values:")
print(missing_df.head(10))

In [ ]:
df = app_data.copy()

numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if 'TARGET' in numerical_cols:
    numerical_cols.remove('TARGET')

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"Numerical columns: {len(numerical_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

num_imputer = SimpleImputer(strategy='median')
df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])

cat_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols])

print("\nMissing values after handling:")
print(df.isnull().sum().sum())

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

print("\nCategorical variables encoded.")

df['INCOME_TO_CREDIT_RATIO'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
df['ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
df['DAYS_EMPLOYED_PERCENT'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(df.median())

print("Feature engineering completed.")

X = df.drop('TARGET', axis=1)
y = df['TARGET']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# Target distribution
plt.figure(figsize=(10, 6))
target_counts = y.value_counts()
plt.bar(['Repaid (0)', 'Default (1)'], target_counts.values)
plt.title('Loan Repayment Status Distribution')
plt.ylabel('Count')
for i, v in enumerate(target_counts.values):
    plt.text(i, v + 1000, str(v), ha='center')
plt.show()

# Correlation with target
correlations = X.corrwith(y).abs().sort_values(ascending=False)
top_correlations = correlations.head(10)

plt.figure(figsize=(12, 8))
top_correlations.plot(kind='bar')
plt.title('Top 10 Features Correlated with Target')
plt.ylabel('Absolute Correlation')
plt.show()

# Distribution of important features by target
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# AMT_INCOME_TOTAL
axes[0, 0].hist(X[y == 0]['AMT_INCOME_TOTAL'].apply(lambda x: min(x, 1000000)), bins=50, alpha=0.7, label='Repaid')
axes[0, 0].hist(X[y == 1]['AMT_INCOME_TOTAL'].apply(lambda x: min(x, 1000000)), bins=50, alpha=0.7, label='Default')
axes[0, 0].set_title('Income Distribution by Repayment Status')
axes[0, 0].set_xlabel('Income (capped at 1M)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].legend()

# AMT_CREDIT
axes[0, 1].hist(X[y == 0]['AMT_CREDIT'].apply(lambda x: min(x, 2000000)), bins=50, alpha=0.7, label='Repaid')
axes[0, 1].hist(X[y == 1]['AMT_CREDIT'].apply(lambda x: min(x, 2000000)), bins=50, alpha=0.7, label='Default')
axes[0, 1].set_title('Credit Amount Distribution by Repayment Status')
axes[0, 1].set_xlabel('Credit Amount (capped at 2M)')
axes[0, 1].set_ylabel('Count')
axes[0, 1].legend()

# DAYS_BIRTH (convert to years)
axes[1, 0].hist((-X[y == 0]['DAYS_BIRTH'] / 365).apply(lambda x: min(x, 70)), bins=50, alpha=0.7, label='Repaid')
axes[1, 0].hist((-X[y == 1]['DAYS_BIRTH'] / 365).apply(lambda x: min(x, 70)), bins=50, alpha=0.7, label='Default')
axes[1, 0].set_title('Age Distribution by Repayment Status')
axes[1, 0].set_xlabel('Age (capped at 70)')
axes[1, 0].set_ylabel('Count')
axes[1, 0].legend()

# INCOME_TO_CREDIT_RATIO
axes[1, 1].hist(X[y == 0]['INCOME_TO_CREDIT_RATIO'].apply(lambda x: min(x, 5)), bins=50, alpha=0.7, label='Repaid')
axes[1, 1].hist(X[y == 1]['INCOME_TO_CREDIT_RATIO'].apply(lambda x: min(x, 5)), bins=50, alpha=0.7, label='Default')
axes[1, 1].set_title('Income to Credit Ratio by Repayment Status')
axes[1, 1].set_xlabel('Income to Credit Ratio (capped at 5)')
axes[1, 1].set_ylabel('Count')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Correlation heatmap of top features
top_features = top_correlations.index.tolist()[:10]
if 'TARGET' in top_features:
    top_features.remove('TARGET')

plt.figure(figsize=(12, 10))
corr_matrix = X[top_features].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap of Top Features')
plt.show()

In [ ]:
print(f"Class distribution before handling: {y.value_counts()}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"Class distribution after SMOTE: {pd.Series(y_train_resampled).value_counts()}")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print("Training models...")

models = {
    'Logistic Regression': LogisticRegression(random_state=42, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(random_state=42, class_weight='balanced'),
    'XGBoost': XGBClassifier(random_state=42, scale_pos_weight=(len(y_train_resampled) - sum(y_train_resampled)) / sum(y_train_resampled)),
    'LightGBM': LGBMClassifier(random_state=42, class_weight='balanced')
}

results = {}
for name, model in models.items():
    print(f"Training {name}...")

    if name in ['Logistic Regression', 'Random Forest']:
        model.fit(X_train_scaled, y_train_resampled)
    else:
        model.fit(X_train_resampled, y_train_resampled)

    if name in ['Logistic Regression', 'Random Forest']:
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"{name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")
    print("-" * 50)

comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[m]['accuracy'] for m in results],
    'Precision': [results[m]['precision'] for m in results],
    'Recall': [results[m]['recall'] for m in results],
    'F1-Score': [results[m]['f1'] for m in results],
    'ROC-AUC': [results[m]['roc_auc'] for m in results]
})

print("\nModel Performance Comparison:")
print(comparison_df)

In [ ]:
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, metric in enumerate(metrics):
    values = [results[m][metric.lower()] for m in results]
    axes[i].bar(results.keys(), values)
    axes[i].set_title(metric)
    axes[i].tick_params(axis='x', rotation=45)

    for j, v in enumerate(values):
        axes[i].text(j, v + 0.01, f'{v:.3f}', ha='center')

if len(metrics) < len(axes):
    axes[-1].set_visible(False)

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))
for name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['y_pred_proba'])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.show()

best_model_name = max(results, key=lambda x: results[x]['f1'])
best_model = results[best_model_name]['model']
y_pred_best = results[best_model_name]['y_pred']

print(f"Best model: {best_model_name}")

plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix - {best_model_name}')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

print(f"\nClassification Report - {best_model_name}:")
print(classification_report(y_test, y_pred_best))

In [ ]:
if best_model_name in ['Logistic Regression', 'Random Forest']:
    explainer = shap.Explainer(best_model, X_train_scaled)
    shap_values = explainer(X_test_scaled)
else:
    explainer = shap.Explainer(best_model, X_train_resampled)
    shap_values = explainer(X_test)

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test, plot_type="bar", max_display=15)
plt.title(f'Global Feature Importance - {best_model_name}')
plt.show()

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test, max_display=15)
plt.title(f'Feature Impact on Model Output - {best_model_name}')
plt.show()

default_indices = np.where(y_pred_best == 1)[0]
if len(default_indices) > 0:
    instance_idx = default_indices[0]
    print(f"Force plot for instance {instance_idx} (predicted as default):")
    
    plt.figure(figsize=(12, 6))
    shap.force_plot(
        explainer.expected_value, 
        shap_values.values[instance_idx], 
        X_test.iloc[instance_idx] if best_model_name in ['XGBoost', 'LightGBM'] else pd.DataFrame(X_test_scaled).iloc[instance_idx],
        matplotlib=True,
        show=False
    )
    plt.title(f'SHAP Force Plot for Instance {instance_idx} - {best_model_name}')
    plt.tight_layout()
    plt.show()

    print(f"\nFeature values for instance {instance_idx}:")
    if best_model_name in ['XGBoost', 'LightGBM']:
        display(X_test.iloc[instance_idx:instance_idx+1])
    else:
        display(pd.DataFrame(X_test_scaled, columns=X.columns).iloc[instance_idx:instance_idx+1])
else:
    print("No instances predicted as default in the test set.")

feature_importance = np.abs(shap_values.values).mean(0)
most_important_feature_idx = np.argmax(feature_importance)
most_important_feature = X.columns[most_important_feature_idx]

print(f"Most important feature: {most_important_feature}")

plt.figure(figsize=(12, 8))
shap.dependence_plot(
    most_important_feature_idx, 
    shap_values.values, 
    X_test if best_model_name in ['XGBoost', 'LightGBM'] else pd.DataFrame(X_test_scaled, columns=X.columns),
    interaction_index=None
)
plt.title(f'Dependence Plot for {most_important_feature} - {best_model_name}')
plt.show()